Есть датфрейм с authors, text, annotation и title. Текст предобработан, стоп-слова удалены. У одной статьи может быть несколько авторов, тогда они записаны в одной ячейке authors и разделены запятой. 

Функции:
- для вычисления всех соавторов одного человека по ФИО, они могут быть записаны в разных ячейках датафрейма. 

- функция для векторизации текста.  

- функция для вычисления коэффициента схожести.

- функция для рекомендации новых соавторов. 

В функцию для рекомендации новых соавторов подаётся на вход ФИО автора. На выходе ФИО автора, названия всех статей в которых он участвовал, список его соавторов, список трёх рекомендуемых новых авторов/коллективов авторов, названия их статей и коэффициент близости. 

Ограничения:

- Нельзя рекомендовать автора самому себе 
- Нельзя рекомендовать автору его текущих соавторов. 

Особенности:

Необходимо предусмотреть ситуацию когда рекомендуется несколько человек, написавших одну и ту же статью, при совпадении коэффициента близости у новых соавторов необходимо объединять их ФИО через запятую, писать их общую статью один раз и на 2 и 3 месте писать новых рекомендуемых авторов. При совпадении 2 и 3 рекомендации также следует их объединять и на третье место выводить следующих по коэффициенту схожести коллектив авторов/автора.

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import pprint

In [3]:
df = pd.read_json('it-st-articles-full-PREPROC.json')
df

,year,title,authors,link,annotation,text
0,2025,актуальный задача математический обеспечение с...,"Головин С.А., Лоцманов А.Н., Тихомиров С.Г.",http://itstd-journal.ru/wp-content/uploads/202...,российский союз промышленник предприниматель р...,": ИТ стандартизация, умный (SMART) стандарт, э..."
1,2025,обзор зарубежный международный опыт стандартиз...,Денисова О.А.,http://itstd-journal.ru/wp-content/uploads/202...,мир меняться невероятный темп наблюдать прорыв...,": цифровая трансформация, SMART-стандарты, CEN..."
2,2025,особенность перспектива применение структура п...,"Быковцев Ю.А., Лохин В.М., Манько С.В., Смирно...",http://itstd-journal.ru/wp-content/uploads/202...,модульный подход построение манипуляционный ро...,": модульный манипуляционный робот, динамическа..."
3,2025,подход одноклассовый классификация регулярный ...,Демидов Н.А.,http://itstd-journal.ru/wp-content/uploads/202...,резюме цель статья рассмотреть задача выявлени...,": регулярное выражение, паттерн новизны, BERT,..."
4,2025,унификация описание программный аппаратный мод...,"Кряхтунов Г.М., Боронников А.С., Платонова О.В.",http://itstd-journal.ru/wp-content/uploads/202...,проектирование базис сверхбольшой интегральный...,": описание, программные модели, аппаратные мод..."
...,...,...,...,...,...,...
335,2015,проблема разработка применение российский феде...,Щербина В.И.,http://itstd-journal.ru/wp-content/uploads/201...,,": переводные стандарты, международные стандарт..."
336,2015,опыт интеграция проблема стандартизация распре...,Куделькин В.А.,http://itstd-journal.ru/wp-content/uploads/201...,рассматриваться опыт разработка интегрировать ...,": мониторинг, распределенные системы, интеграц..."
337,2014,путь обеспечение безопасность функционирование...,Петров А.Б.; Андрианова Е.Г.; Сычева А.И.; Баг...,http://itstd-journal.ru/wp-content/uploads/201...,,": безопасность функционирования, информационны..."
338,2014,развитие медицинский информатика новый направл...,Борисова Н.В.; Карачунский А.И.; Старичкова Ю.В.,http://itstd-journal.ru/wp-content/uploads/201...,,": медицинская информатика, здравоохранение, ин..."


In [5]:
#df['authors'] = df['authors'].str.split(',') #авторов через запятую разделяем

Функция для поиска всех соавторов:

In [35]:
def get_coauthors(df, author_name):
    coauthors = set()
    for authors_list in df['authors'].str.split(', '):
        if author_name in authors_list:
            coauthors.update(authors_list)
    coauthors.discard(author_name)  # удаляем самого автора
    return list(coauthors)

get_coauthors(df, 'Тест А.Б.')

['Тест Е.Д.', 'Тест В.Г.']

Векторизация текста:

In [5]:
import re

def find_coauthors(df, author):
    coauthors = set()
    # Паттерн для разделения по пробелу после точки, перед заглавной буквой
    split_pattern = re.compile(r'(?<=\.)\s+(?=[А-ЯЁ])')
    
    for authors_str in df['authors'].dropna():
        # Нормализуем разделители: заменяем ; и , на специальный символ |
        normalized = re.sub(r'[;,]', '|', authors_str)
        # Разбиваем по | и удаляем пустые строки
        parts = [p.strip() for p in normalized.split('|') if p.strip()]
        
        authors_list = []
        for part in parts:
            # Делим часть на авторов по паттерну
            split_part = split_pattern.split(part)
            authors_list.extend([a.strip() for a in split_part if a.strip()])
        
        # Проверяем наличие автора и собираем соавторов
        if author in authors_list:
            for coauthor in authors_list:
                if coauthor != author:
                    coauthors.add(coauthor)
    
    return sorted(coauthors)

Схожесть между целевыми статьями и всеми остальными

In [8]:
def calculate_similarity(tfidf_matrix, target_indices, all_indices):
    similarities = cosine_similarity(tfidf_matrix[target_indices], tfidf_matrix[all_indices])
    return similarities.max(axis=0)

Основная функция с рекомендацией

In [29]:
def recommend_new_coauthors(df, author_name):
    # Нормализуем имя автора для сравнения
    author_name = author_name.strip()
    
    # Находим все статьи автора
    target_mask = df['authors'].apply(lambda x: author_name in [a.strip() for a in x.split(',')])
    target_indices = df[target_mask].index.tolist()
    
    if not target_indices:
        return {"error": "Автор не найден"}
    
    current_coauthors = find_coauthors(df, author_name)
    
    # Векторизация текстов
    tfidf_matrix, _ = vectorize_texts(df['text'])
    
    all_indices = df.index.tolist()
    similarities = calculate_similarity(tfidf_matrix, target_indices, all_indices)
    
    results = pd.DataFrame({
        'index': all_indices,
        'similarity': similarities,
        'authors': df['authors'],
        'title': df['title']
    })
    
    # Улучшенная фильтрация - исключаем все статьи, где автор является соавтором
    results = results[
        (~results['authors'].apply(lambda x: author_name in [a.strip() for a in x.split(',')])) &
        (results['similarity'] < 0.99)  # Добавляем небольшой запас для схожести
    ]
    
    # Группировка по коэффициенту схожести
    grouped = results.groupby('similarity').agg({
        'authors': lambda x: ', '.join(sorted(set(', '.join(x).split(', ')))),
        'title': lambda x: ' | '.join(sorted(set(x)))  # Разделитель для ясности
    }).reset_index().sort_values('similarity', ascending=False)
    
    recommendations = []
    used_authors = set(current_coauthors + [author_name])
    
    for _, row in grouped.iterrows():
        candidates = [a.strip() for a in row['authors'].split(',') 
                    if a.strip() not in used_authors and a.strip() != author_name]
        
        if candidates:
            unique_candidates = []
            for candidate in candidates:
                if candidate not in used_authors:
                    unique_candidates.append(candidate)
                    used_authors.add(candidate)
            
            if unique_candidates:
                recommendations.append({
                    'authors': ', '.join(unique_candidates),
                    'titles': row['title'],
                    'similarity': round(row['similarity'], 2)
                })
        
        if len(recommendations) >= 3:
            break
    
    output = {
        "Автор": author_name,
        "Текущие соавторы": current_coauthors,
        "Статьи автора": df.loc[target_indices, 'title'].tolist(),
        "Рекомендуемые соавторы": recommendations[:3]
    }
    
    return output

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

def vectorize_texts(texts):
    # Создаем TF-IDF векторизатор с базовыми параметрами
    vectorizer = TfidfVectorizer(
        stop_words='english',  # удаляем стоп-слова
        max_features=5000,     # ограничиваем количество фичей
        ngram_range=(1, 2)     # учитываем униграммы и биграммы
    )
    
    # Применяем векторизацию
    tfidf_matrix = vectorizer.fit_transform(texts)
    
    return tfidf_matrix, vectorizer

In [27]:
rec_list = recommend_new_coauthors(df, 'Андрианова Е.Г.')
pprint.pprint(rec_list)

{'Автор': 'Андрианова Е.Г.',
 'Рекомендуемые соавторы': [{'authors': 'АнтоновА.В., Сидорин В.В.',
                             'similarity': 0.55,
                             'titles': 'стандарт виртуальный аудит сервисный '
                                       'центр обслуживание продукция '
                                       'потребитель'},
                            {'authors': 'Зуев А.С., Исаев Р. А., Салямов Р.Р',
                             'similarity': 0.38,
                             'titles': 'исследование подход создание набор '
                                       'данные основа временной ряд '
                                       'регистрировать работа канальный '
                                       'нейроинтерфейс'},
                            {'authors': 'Болбаков Р.Г., Куликов А.А., '
                                        'Плотников С.Б., Тигай М.',
                             'similarity': 0.36,
                             'titles': 'эмерджентный